In [1]:
import logging
import os
import re
import warnings
from typing import Any

import cv2
import numpy as np
import pandas as pd
import skimage as ski
import torch
from catboost import CatBoostClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from sklearn.exceptions import UndefinedMetricWarning
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import (
    accuracy_score,
    cohen_kappa_score,
    f1_score,
    make_scorer,
    precision_score,
    recall_score,
)
from sklearn.metrics._scorer import _BaseScorer
from sklearn.model_selection import cross_validate
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from tqdm.auto import tqdm
from transformers import BertModel, BertTokenizer
from xgboost import XGBClassifier

In [2]:
ROOT = "../data/"
DATASET = os.path.join(ROOT, "br-pt-narrative-essays.csv")
SCORING = {
    "acc": make_scorer(accuracy_score),
    "macro_prec": make_scorer(precision_score, average="macro"),
    "weighted_prec": make_scorer(precision_score, average="weighted"),
    "macro_recall": make_scorer(recall_score, average="macro"),
    "weighted_recall": make_scorer(recall_score, average="weighted"),
    "macro_f1": make_scorer(f1_score, average="macro"),
    "weighted_f1": make_scorer(f1_score, average="weighted"),
    "kappa": make_scorer(cohen_kappa_score)
}
CV = 5
SEED = 42

In [3]:
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

In [4]:
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
)
logger = logging.getLogger(__name__)

In [5]:
tqdm.pandas()

In [6]:
class BERTEncoder(BaseEstimator, TransformerMixin):
    def __init__(
        self,
        model_name="neuralmind/bert-base-portuguese-cased",
        max_length=128,
    ):
        self.model_name = model_name
        self.tokenizer = BertTokenizer.from_pretrained(model_name)
        self.model = BertModel.from_pretrained(model_name)
        self.max_length = max_length

    def transform(self, X, y=None):
        """Tokenize and encode the text, and get CLS token embedding for classification tasks"""

        X = list(X)

        if not isinstance(X, list):
            raise ValueError("Not a list of strings")
        elif not all(isinstance(x, str) for x in X):
            raise ValueError("Not all instances are strings.")

        inputs = self.tokenizer(
            X,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=self.max_length,
        )
        with torch.no_grad():
            outputs = self.model(**inputs)
        return outputs.last_hidden_state[:, 0, :].numpy()

    def fit(self, X, y=None):
        return self


class LBPEncoder(BaseEstimator, TransformerMixin):
    """LBP encoder for image data."""

    def __init__(self, radius: int = 1, sampling_pixels: int = 106):
        self.radius = radius
        self.sampling_pixels = sampling_pixels

    def transform(self, X, y=None):
        """Extract the LBP from the images batch."""
        logger.debug("Encoding images...")
        X = list(X)
        logger.debug("Converting...")
        cvt_imgs = [self._cvt(img) for img in X]
        logger.debug("Running LBP algorithm...")
        imgs_lbps = [self._get_lbp(img) for img in cvt_imgs]
        logger.debug("Getting the histograms...")
        imgs_hists = [self._get_hist(img_lbp) for img_lbp in imgs_lbps]
        logger.debug("Extracting LBP features from histograms...")
        features = self._get_features(imgs_hists)
        logger.debug("Finished with encoding images")

        return features

    def fit(self, X, y=None):
        return self
    
    def _cvt(self, img):
        if len(img.shape) > 2:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            _, img = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

        i_min = np.min(img)
        i_max = np.max(img)
        if i_max - i_min != 0:
            img = (img - i_min) / (i_max - i_min)

        img = img.astype(np.uint8)

        return img

    def _get_lbp(self, img):
        lbp = ski.feature.local_binary_pattern(
            img, self.sampling_pixels, self.radius, method="uniform"
        )
        return (img, lbp)

    def _get_hist(self, img_lbp):
        img, lbp = img_lbp
        hist, _ = np.histogram(
            lbp.ravel(),
            bins=np.arange(0, self.sampling_pixels + 3),
            range=(0, self.sampling_pixels + 2),
        )
        hist = hist.astype("float")
        hist /= hist.sum() + 1e-6
        return img, hist
    
    def _get_features(self, imgs_hists):
        hists = [img_hist[1] for img_hist in imgs_hists]
        features = []
        for h in hists:
            features.extend(h)
        return hists

In [7]:
def _create_tfidf_pipeline(clf: BaseEstimator) -> Pipeline:
    pipeline = Pipeline(
        [
            ("encoder", TfidfVectorizer()),
            ("clf", clf),
        ]
    )
    return pipeline


def _create_lbp_pipeline(clf: BaseEstimator) -> Pipeline:
    pipeline = Pipeline(
        [
            ("encoder", LBPEncoder()),
            ("clf", clf),
        ]
    )
    return pipeline


def _create_bert_pipeline(clf: BaseEstimator) -> Pipeline:
    pipeline = Pipeline([("encoder", BERTEncoder()), ("clf", clf)])
    return pipeline

In [8]:
def scores_to_df(scores: tuple[tuple[str, str, float]]) -> pd.DataFrame:
    """Transform the scores dictionary into a dataframe object."""
    data = []
    for score in scores:
        data.append(
            {
                "algo": score["algo"],
                "encoder": score["encoder"],
                "acc": np.mean(score["test_acc"]),
                "macro_prec": np.mean(score["test_macro_prec"]),
                "weighted_prec": np.mean(score["test_weighted_prec"]),
                "macro_recall": np.mean(score["test_macro_recall"]),
                "weighted_recall": np.mean(score["test_weighted_recall"]),
                "macro_f1": np.mean(score["test_macro_f1"]),
                "weighted_f1": np.mean(score["test_weighted_f1"]),
                "kappa": np.mean(score["test_kappa"]),
            }
        )
    df = pd.DataFrame(data=data)
    return df


def clean_text(text: str) -> str:
    """Remove unwanted tokens from the text."""
    text = re.sub(r"\n", "", text)
    return text


def transform_image(image):
    resized_image = cv2.resize(image, (100, 100))
    gray_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)
    _, binary_image = cv2.threshold(gray_image, 127, 255, cv2.THRESH_BINARY)
    return binary_image


def run_experiments(
    clfs: list[BaseEstimator],
    df: pd.DataFrame,
    scoring: dict[str, _BaseScorer],
    cv: int,
    preprocessors: list[Any],
    text_feature_column: str,
    image_feature_column: str,
    target_column: str,
    n_jobs: int = 1,
):
    """Run Scikit-Learn based experiments."""

    ALGO_NAME_DICT = {
        ExtraTreesClassifier: "Extra Trees",
        DecisionTreeClassifier: "Decision Tree",
        SVC: "Support Vector",
        RandomForestClassifier: "Random Forest",
        CatBoostClassifier: "CatBoost",
        XGBClassifier: "XGBoost",
        MLPClassifier: "MLP"
        
    }

    ENCODER_NAME_DICT = {
        TfidfVectorizer: "TF-IDF",
        BERTEncoder: "BERT",
        LBPEncoder: "LBP"
    }

    def _run_sklearn_cv(
        clf: BaseEstimator,
        X: pd.DataFrame,
        y: pd.DataFrame,
        scoring: dict[str, _BaseScorer],
        cv: int,
    ):
        """Run cross-validation on a Scikit-Learn estimator."""

        assert issubclass(type(clf), BaseEstimator) is True

        cv_results = cross_validate(
            clf,
            X,
            y,
            cv=cv,
            scoring=scoring,
            n_jobs=n_jobs,
        )
        cv_results["algo"] = ALGO_NAME_DICT[type(clf.named_steps["clf"])]
        cv_results["encoder"] = ENCODER_NAME_DICT[type(clf.named_steps["encoder"])]
        return cv_results

    cv_results_list = []

    # Apply preprocessors
    for preprocessor in preprocessors["image"]:
        df[image_feature_column] = df[image_feature_column].apply(
            lambda feature: preprocessor(cv2.imread(feature))
        )

    for preprocessor in preprocessors["text"]:
        df[text_feature_column] = df[text_feature_column].apply(
            lambda feature: preprocessor(feature)
        )

    label_encoder = LabelEncoder()
    df[target_column] = label_encoder.fit_transform(df[target_column])

    for i, clf in enumerate(tqdm(clfs, desc="Classifiers")):
        clf_name = ALGO_NAME_DICT[type(clf)]
        logger.debug(f"#{i + 1} classifier: {clf_name}")

        # Create pipelines
        tfidf_pipeline = _create_tfidf_pipeline(clf)
        bert_pipeline = _create_bert_pipeline(clf)
        lbp_pipeline = _create_lbp_pipeline(clf)

        # Run CV using the pipelines
        logger.debug(f"Running TF-IDF pipeline #{i + 1}")
        tfidf_cv_results = _run_sklearn_cv(
            tfidf_pipeline,
            df[text_feature_column],
            df[target_column],
            scoring,
            cv,
        )
        logger.debug(f"Ending TF-IDF pipeline #{i + 1}")

        bert_cv_results = _run_sklearn_cv(
            bert_pipeline,
            df[text_feature_column],
            df[target_column],
            scoring,
            cv,
        )

        logger.debug(f"Running LBP pipeline #{i + 1}")
        lbp_cv_results = _run_sklearn_cv(
            lbp_pipeline,
            df[image_feature_column],
            df[target_column],
            scoring,
            cv,
        )
        logger.debug(f"Ending LBP pipeline #{i + 1}")

        # Append results
        cv_results_list.append(tfidf_cv_results)
        cv_results_list.append(bert_cv_results)
        cv_results_list.append(lbp_cv_results)

    return cv_results_list

In [9]:
df = pd.read_csv(DATASET, index_col=0)
df.head()

,text,motivating_situation,image_url,image_path,cohesion,thematic_coherence,formal_register,text_typology
0,( O chorrinho nino )\n-\nEu e...,Eu encontrei em cima do armário alguns potes c...,https://storage.googleapis.com/ciclos-10698-bu...,MEC/Rc7dMxTP7ZdLNEvmF0jo/iet1QFw2ARNkv9Hx6KGe/...,3,3,3,4
1,.As meninas do potes de Tintas [T]\nUma vez eu...,Eu encontrei em cima do armário alguns potes c...,https://storage.googleapis.com/ciclos-10698-bu...,MEC/Rc7dMxTP7ZdLNEvmF0jo/F80gTOBoh2Lk5JtrLWUO/...,3,3,3,4
2,1 uma menina encontrou varios potes de tinta ...,Eu encontrei em cima do armário alguns potes c...,https://storage.googleapis.com/ciclos-10698-bu...,MEC/Rc7dMxTP7ZdLNEvmF0jo/HjnamZPzaZcDeEANF2IC/...,3,3,3,4
3,2011 [T] A menina da (artes). \n\nDePois que e...,Eu encontrei em cima do armário alguns potes c...,https://storage.googleapis.com/ciclos-10698-bu...,MEC/Rc7dMxTP7ZdLNEvmF0jo/r3DUtFJn9twSvqcNloNM/...,3,3,3,4
4,28/03/2022\n[T] A cobra felena\nem uma tarde m...,Eu encontrei em cima do armário alguns potes c...,https://storage.googleapis.com/ciclos-10698-bu...,MEC/Rc7dMxTP7ZdLNEvmF0jo/tKZIn59KtOcbe9z0n6oY/...,4,1,4,4


In [10]:
df["image_path"] = df["image_path"].apply(lambda image_path: os.path.join(ROOT, image_path))
df.head()

,text,motivating_situation,image_url,image_path,cohesion,thematic_coherence,formal_register,text_typology
0,( O chorrinho nino )\n-\nEu e...,Eu encontrei em cima do armário alguns potes c...,https://storage.googleapis.com/ciclos-10698-bu...,../data/MEC/Rc7dMxTP7ZdLNEvmF0jo/iet1QFw2ARNkv...,3,3,3,4
1,.As meninas do potes de Tintas [T]\nUma vez eu...,Eu encontrei em cima do armário alguns potes c...,https://storage.googleapis.com/ciclos-10698-bu...,../data/MEC/Rc7dMxTP7ZdLNEvmF0jo/F80gTOBoh2Lk5...,3,3,3,4
2,1 uma menina encontrou varios potes de tinta ...,Eu encontrei em cima do armário alguns potes c...,https://storage.googleapis.com/ciclos-10698-bu...,../data/MEC/Rc7dMxTP7ZdLNEvmF0jo/HjnamZPzaZcDe...,3,3,3,4
3,2011 [T] A menina da (artes). \n\nDePois que e...,Eu encontrei em cima do armário alguns potes c...,https://storage.googleapis.com/ciclos-10698-bu...,../data/MEC/Rc7dMxTP7ZdLNEvmF0jo/r3DUtFJn9twSv...,3,3,3,4
4,28/03/2022\n[T] A cobra felena\nem uma tarde m...,Eu encontrei em cima do armário alguns potes c...,https://storage.googleapis.com/ciclos-10698-bu...,../data/MEC/Rc7dMxTP7ZdLNEvmF0jo/tKZIn59KtOcbe...,4,1,4,4


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1188 entries, 0 to 1187
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   text                  1188 non-null   object
 1   motivating_situation  1188 non-null   object
 2   image_url             1188 non-null   object
 3   image_path            1188 non-null   object
 4   cohesion              1188 non-null   int64 
 5   thematic_coherence    1188 non-null   int64 
 6   formal_register       1188 non-null   int64 
 7   text_typology         1188 non-null   int64 
dtypes: int64(4), object(4)
memory usage: 83.5+ KB


In [12]:
def remove_truncated_images(df, column_name):
    """
    Remove rows with truncated or corrupted images from the DataFrame.

    Parameters:
    df (pd.DataFrame): DataFrame containing image file paths.
    column_name (str): Column name containing image file paths.

    Returns:
    pd.DataFrame: DataFrame with rows containing truncated or corrupted images removed.
    """
    # Function to check if an image is valid
    def is_valid_image(image_path):
        with open(image_path, 'rb') as image_file:
            check_chars = image_file.read()[-2:]
        if check_chars != b'\xff\xd9':
            return False
        return True

    # Apply the function to the DataFrame and filter out invalid images
    valid_image_paths = df[column_name].apply(is_valid_image)
    filtered_df = df[valid_image_paths].reset_index(drop=True)

    return filtered_df

df = remove_truncated_images(df, "image_path")

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1163 entries, 0 to 1162
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   text                  1163 non-null   object
 1   motivating_situation  1163 non-null   object
 2   image_url             1163 non-null   object
 3   image_path            1163 non-null   object
 4   cohesion              1163 non-null   int64 
 5   thematic_coherence    1163 non-null   int64 
 6   formal_register       1163 non-null   int64 
 7   text_typology         1163 non-null   int64 
dtypes: int64(4), object(4)
memory usage: 72.8+ KB


In [14]:
# df["image"] = df["image_path"].progress_apply(lambda image_path: transform_image(cv2.imread(image_path)))

In [15]:
# lbp_encoder = LBPEncoder()
# features = lbp_encoder.fit_transform(df["image"])

In [16]:
clfs = [
    SVC(random_state=SEED),
    RandomForestClassifier(random_state=SEED),
    DecisionTreeClassifier(random_state=SEED),
    ExtraTreesClassifier(random_state=SEED),
    XGBClassifier(
        objective="multi:softprob",
        eval_metric="mlogloss",
    ),
    MLPClassifier(hidden_layer_sizes=(100,), max_iter=300, random_state=SEED),
]

test_df = df.sample(n=100)

with warnings.catch_warnings():
    warnings.simplefilter(action="ignore", category=UndefinedMetricWarning)
    warnings.simplefilter(action="ignore", category=FutureWarning)
    warnings.simplefilter(action="ignore", category=UserWarning)
    preprocessors = {"text": [clean_text], "image": [transform_image]}
    cv_results = run_experiments(
        clfs,
        test_df,
        scoring=SCORING,
        cv=CV,
        preprocessors=preprocessors,
        text_feature_column="text",
        image_feature_column="image_path",
        target_column="cohesion",
    )

Classifiers:   0%|          | 0/6 [00:00<?, ?it/s]

In [17]:
cv_results_df = scores_to_df(cv_results)
cv_results_df

,algo,encoder,acc,macro_prec,weighted_prec,macro_recall,weighted_recall,macro_f1,weighted_f1,kappa
0,Support Vector,TF-IDF,0.68,0.170000,0.463000,0.250000,0.68,0.202317,0.550731,0.000000
1,Support Vector,BERT,0.68,0.170000,0.463000,0.250000,0.68,0.202317,0.550731,0.000000
2,Support Vector,LBP,0.68,0.170000,0.463000,0.250000,0.68,0.202317,0.550731,0.000000
3,Random Forest,TF-IDF,0.66,0.168333,0.458333,0.242857,0.66,0.198641,0.540437,-0.022222
4,Random Forest,BERT,0.69,0.274722,0.525500,0.288095,0.69,0.262641,0.584128,0.081332
5,Random Forest,LBP,0.62,0.168586,0.459408,0.228022,0.62,0.193485,0.526758,-0.030699
6,Decision Tree,TF-IDF,0.56,0.348643,0.587613,0.329304,0.56,0.327667,0.561300,0.139787
7,Decision Tree,BERT,0.50,0.315278,0.578222,0.324084,0.50,0.294994,0.508523,0.116069
8,Decision Tree,LBP,0.48,0.225863,0.482208,0.228755,0.48,0.222548,0.471664,0.003547
9,Extra Trees,TF-IDF,0.66,0.168333,0.458333,0.242857,0.66,0.198641,0.540437,-0.022222


In [18]:
print(cv_results_df.to_latex())

\begin{tabular}{lllrrrrrrrr}
\toprule
 & algo & encoder & acc & macro_prec & weighted_prec & macro_recall & weighted_recall & macro_f1 & weighted_f1 & kappa \\
\midrule
0 & Support Vector & TF-IDF & 0.680000 & 0.170000 & 0.463000 & 0.250000 & 0.680000 & 0.202317 & 0.550731 & 0.000000 \\
1 & Support Vector & BERT & 0.680000 & 0.170000 & 0.463000 & 0.250000 & 0.680000 & 0.202317 & 0.550731 & 0.000000 \\
2 & Support Vector & LBP & 0.680000 & 0.170000 & 0.463000 & 0.250000 & 0.680000 & 0.202317 & 0.550731 & 0.000000 \\
3 & Random Forest & TF-IDF & 0.660000 & 0.168333 & 0.458333 & 0.242857 & 0.660000 & 0.198641 & 0.540437 & -0.022222 \\
4 & Random Forest & BERT & 0.690000 & 0.274722 & 0.525500 & 0.288095 & 0.690000 & 0.262641 & 0.584128 & 0.081332 \\
5 & Random Forest & LBP & 0.620000 & 0.168586 & 0.459408 & 0.228022 & 0.620000 & 0.193485 & 0.526758 & -0.030699 \\
6 & Decision Tree & TF-IDF & 0.560000 & 0.348643 & 0.587613 & 0.329304 & 0.560000 & 0.327667 & 0.561300 & 0.139787 \\
7 & Decisi